## Loading Data to prepare for Topic Modeling

This is the first notebook in the series of Topic Modeling notebooks

Please note: open Jupyter Notebooks using Anaconda Prompt: jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10
You will need this increaset data rate for NLP Data Processing. See more here: https://stackoverflow.com/questions/43288550/iopub-data-rate-exceeded-in-jupyter-notebook-when-viewing-image

In [ ]:
## General Dependencies
import re
import numpy as np
import pandas as pd
from pprint import pprint
import sys, os
import glob
from tika import parser # pip install tika
import inspect
import datetime
import pickle5 as pickle

## Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim import models
#from gensim.models.coherencemodel import CoherenceModel
from gensim.models import CoherenceModel
from gensim.models import LdaModel
from gensim.models.wrappers import LdaMallet
from gensim.models import ldaseqmodel


## Preprocessing
import spacy
import nltk as nltk
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

## Plotting
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import ast

## Other Libraries
from operator import itemgetter

## ScikitLearn
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

### Opening data: if data are a folder of full text PDFs

In [ ]:
## Use the glob method to retrieve files/pathnames in the directory
## https://www.geeksforgeeks.org/how-to-use-glob-function-to-find-files-recursively-in-python/

directory = "data/Interview_transcripts"

# ## Other file directories
# ## "data/10_News_All_Final"
# ## "data/1_News_Industry"
# ## "data/2_Government_Documents"
# ## "data/3_Altmetric_Policy"

files = list(glob.glob(os.path.join(directory,'*.*')))

## Inspect output as needed
print(files)

## Other Resources
## https://stackoverflow.com/questions/34000914/how-to-create-a-list-from-filenames-in-a-user-specified-directory-in-python
## https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory
## https://stackoverflow.com/questions/33912773/python-read-txt-files-into-a-dataframe

In [ ]:
## Extract text from the pdfs and add them to a list using Tika Python
## The output is a dictionary with: metadata, content, status

document_list = []
for f in files:
    raw = parser.from_file(f)
    document_list.append(raw)
    
## Resources
## https://www.geeksforgeeks.org/parsing-pdfs-in-python-with-tika/
## https://stackoverflow.com/questions/34837707/how-to-extract-text-from-a-pdf-file

In [ ]:
## Add the dictionary to a pandas dataframe

text_df = pd.DataFrame(document_list)

## Inspect the output as needed
# text_df.head()
# print(text_df["metadata"][1])
# print(text_df["content"][1])
# text_df.to_csv('gensim_test.csv')

In [ ]:
## Count the number of rows in the dataframe

text_df_count_row = text_df.shape[0]  # gives number of row count

## Inspect output as needed
print(text_df_count_row)

In [ ]:
## Create a new column in the dataframe called "title" and populate it with the title from the metadata key called dc:title

text_df['title'] = [value.get('dc:title') for value in text_df["metadata"]]

## Inspect output as needed
text_df.head()

## Resources
## https://stackoverflow.com/questions/44218812/pandas-add-columns-to-a-dataframe-based-in-dict-from-one-of-the-columns

## If needed, you can review the contents of the metadata column using this code:
## print(text_df['metadata'])
## my_dict.keys()[0]     -> key of "first" element
## my_dict.values()[0]   -> value of "first" element
## my_dict.items()[0]    -> (key, value) tuple of "first" element
##list(contacts.items())[0]

# for v in text_df['metadata']:
#     new = list(v.items())[1]
#     print(new)
   

In [ ]:
## Create a new column in the dataframe called "file_name" and populate it with the title from the metadata key called "resourceName"

text_df['file_name'] = [value.get('resourceName') for value in text_df["metadata"]]

## Inspect output as needed

text_df.head()

## Resources
## https://stackoverflow.com/questions/44218812/pandas-add-columns-to-a-dataframe-based-in-dict-from-one-of-the-columns

## If needed, you can review the contents of the metadata column using this code:
## print(text_df['metadata'])
## my_dict.keys()[0]     -> key of "first" element
## my_dict.values()[0]   -> value of "first" element
## my_dict.items()[0]    -> (key, value) tuple of "first" element
##list(contacts.items())[0]

# for v in text_df['metadata']:
#     new = list(v.items())[1]
#     print(new)

In [ ]:
## Change the file type of the file_name column to string (if it isn't already). 

text_df['file_name'] = text_df['file_name'].astype(str)

## Inspect output as needed
text_df.head()
print(type(text_df['file_name'][1]))
print(text_df.dtypes)

In [ ]:
## Remove the b' before the string name 
text_df['file_name'] = text_df['file_name'].str.strip("b\'\"")

## Inspect output as needed
text_df.head()

## Other method
## text_df['file_name'] = text_df['file_name'].str.decode('utf-8')
## text_df.head()

## Resources
## https://stackoverflow.com/questions/46696679/removing-b-from-string-column-in-a-pandas-dataframe
## https://stackoverflow.com/questions/61970212/pandas-decoding-a-string-returns-nan

In [ ]:
## Count the number of rows in the dataframe

text_df_count_row = text_df.shape[0]  # gives number of row count
print(text_df_count_row)

In [ ]:
## This code was written to inspect the "file_name" column, but is no longer needed for this workflow. 
## Sort the dataframe by values in the "file_name" column
## Notice that some file_names have .pdf or other file type included...this will cause problems
## text_df = text_df.sort_values(by=['file_name'], ascending=False)
## text_df.head()
## Resources
## https://stackoverflow.com/questions/37787698/how-to-sort-pandas-dataframe-from-one-column

## Inspect the output as needed
# print(text_df['file_name'][0])
# print(text_df['file_name'][1])
# print(text_df['file_name'][2])
# print(text_df['file_name'][3])

### Opening data: if data is a pickle file

In [ ]:
## Check working directory
# cwd = os.getcwd()
# print(cwd)

In [ ]:
## path_to_protocol5 = "C:\\Users\\keg827\\AppData\\Local\\Continuum\\anaconda3\\Lib\\site-packages\\pickle5"
## https://www.geeksforgeeks.org/dataframe-read_pickle-method-in-pandas/
# with open(path_to_protocol5, "rb") as fh:
# altmetric_policy_df = pd.read_pickle(r'C:\Users\keg827\Documents\10_Github_Repos\CARDIAimpactanalysis\data\policy_fulltext (1).pkl', compression=None)
   

In [ ]:
## Check recursion depth
## https://riptutorial.com/python/example/17855/increasing-the-maximum-recursion-depth
# def cursing(depth):
#   try:
#     cursing(depth + 1) # actually, re-cursing
#   except RuntimeError as RE:
#     print('I recursed {} times!'.format(depth))
# cursing(0)

In [ ]:
# import sys
# print(sys.getrecursionlimit())

In [ ]:
# sys.setrecursionlimit(4000)

In [ ]:
# filename = "C:\\Users\\keg827\\Documents\\10_Github_Repos\\CARDIAimpactanalysis\\data\\policy_fulltext (1).pkl"
# infile = open(filename,'rb')
# new_dict = pickle.load(infile)
# infile.close()

In [ ]:
# print(type(new_dict))
# new_dict.head()

In [ ]:
## Upload Altmetric data
## pmid_list = pd.read_csv(r"C:\Users\keg827\Documents\10_Github_Repos\pubmedbiopython\cardiaids.csv", encoding= 'unicode_escape')
# altmetric_policy_df = pd.read_csv(r"C:\Users\keg827\Documents\10_Github_Repos\CARDIAimpactanalysis\data\policy_fulltext.csv")
# altmetric_policy_df.head()

## print(altmetric_policy_df['metadata'][1])

### Get Creation Date from Metadata column (if it exists)

In [ ]:
## Create a new column called metadata2 which is a reformatted copy of the metadata column from string to object

# text_df['metadata2']=text_df['metadata'].apply(ast.literal_eval)

## Resource(s)
## https://stackoverflow.com/questions/56102724/how-to-convert-string-representation-of-dictionary-in-pandas-dataframe-to-a-new


In [ ]:
## Extract the creation-date data from the metadata2 column and create a new column called date

# text_df['date'] = [value.get('Creation-Date') for value in text_df["metadata2"]]
# text_df.head()

In [ ]:
## Create a new column called year and extract the year from the date column

## efficient way to extract year from string format date
# text_df['year'] = pd.DatetimeIndex(text_df['date']).year
# text_df.head(50)

### Add Creation Date from outside CSV (if not in PDF metadata)

In [ ]:
## Import dataframe from non-core spreadsheet with year column
# date_df = pd.read_csv("data/10_News_All_Final_2.csv", encoding='ISO-8859-1') 
# date_df.head()
## print(date_df['File_Name'][4])
## date_df.keys()

## print(type(date_df['File_Name'][1]))
## print(date_df.dtypes)

In [ ]:
## Count the number of rows in the dataframe

# date_df_count_row = date_df.shape[0]  # gives number of row count
# print(date_df_count_row)

In [ ]:
## Match text_df and date_df based on title and Metadata_title to add year to the text_df dataframe

# merged_df= text_df.merge(date_df, left_on = 'title', right_on = 'Metadata_Title', how = 'inner')
# merged_df.keys()
# merged_df.head()

## Left Dataframe is text_df
## Right Dataframe is date_df

## Note: An inner join only merges together what the two spreadsheets have in common. Anything not in common will be dropped. 

## Resources
## https://stackoverflow.com/questions/49890305/match-two-columns-from-two-dataframes-and-add-items-from-a-third-column-if-cells
# miscset = miscset.merge(oset, left_on='subset', right_on='some_items', 
#     how='inner').drop(columns='some_items')

In [ ]:
## CANNOT USE THIS because file_name and File_Name are not matchable in this dataset
## Match text_df and date_df based on file_name and File_Name to add year to the text_df dataframe

# merged_df= text_df.merge(date_df, left_on = 'file_name', right_on = 'File_Name', how = 'inner')
# merged_df.keys()
# merged_df.head()

## Left Dataframe is text_df
## Right Dataframe is date_df

## Note: An inner join only merges together what the two spreadsheets have in common. Anything not in common will be dropped. 

## Resources
## https://stackoverflow.com/questions/49890305/match-two-columns-from-two-dataframes-and-add-items-from-a-third-column-if-cells
# miscset = miscset.merge(oset, left_on='subset', right_on='some_items', 
#     how='inner').drop(columns='some_items')

In [ ]:
## Count the number of rows in the dataframe

# merged_df_count_row = merged_df.shape[0]  # gives number of row count
# print(merged_df_count_row)

### Identify and remove duplicates

In [ ]:
## Identify any duplicates in the dataframe by Metadata_Title, Source_title, and Year

# duplicate_df = merged_df[merged_df.duplicated(subset=['Metadata_Title','Source_title ', 'Year'], keep=False)]
# duplicate_df.head()

## Count the number of rows in the dataframe

# duplicate_df_count_row = duplicate_df.shape[0]  # gives number of row count
# print(duplicate_df_count_row)

## Note that some files may have multiple duplicates

In [ ]:
## Drop duplicates from the merged_df using the "Metadata_Title" AND "Source_title" and 'Year'...and keep the "last record" 

# deduplicate_df = merged_df.drop_duplicates(['Metadata_Title','Source_title ', 'Year'], keep= 'last')
# print(deduplicate_df)

## Count the number of rows in the dataframe

# deduplicate_df_count_row = deduplicate_df.shape[0]  # gives number of row count
# print(deduplicate_df_count_row)

## Resources
## https://www.geeksforgeeks.org/python-pandas-dataframe-drop_duplicates/
## https://www.codegrepper.com/code-examples/python/find+duplicated+rows+with+respect+to+multiple+columns+pandas
## https://stackoverflow.com/questions/32093829/remove-duplicates-from-dataframe-based-on-two-columns-a-b-keeping-row-with-max

In [ ]:
## Count the number of duplicates removed from the dataframe

# duplicates_removed = merged_df_count_row - deduplicate_df_count_row
# print(duplicates_removed)

In [ ]:
## Find any missing data in the dataframe
## If any column is mising data, the number of rows with missing data will be reported in this series

# deduplicate_df.isnull().sum(axis = 0)

## Resources: 
## https://stackoverflow.com/questions/15943769/how-do-i-get-the-row-count-of-a-pandas-dataframe
## https://stackoverflow.com/questions/46864740/selecting-a-subset-using-dropna-to-select-multiple-columns

In [ ]:
## Drop rows where values in these columns are NaN

# first_drop_df = deduplicate_df.dropna(subset=['content'], how = 'all')
# text_content_df = first_drop_df.dropna(subset=['Year'], how = 'all')
# text_content_df.head()

In [ ]:
## Count the number of rows in the dataframe

# text_content_df_count_row = text_content_df.shape[0]  # gives number of row count
# print(text_content_df_count_row)

## Find any missing data in the dataframe
## If any column is mising data, the number of rows with missing data will be reported in this series

# text_content_df.isnull().sum(axis = 0)

## Resources: 
## https://stackoverflow.com/questions/15943769/how-do-i-get-the-row-count-of-a-pandas-dataframe
## https://stackoverflow.com/questions/46864740/selecting-a-subset-using-dropna-to-select-multiple-columns

In [ ]:
## Make sure to sort the dataframe by YEAR and reset the index. 
## This is VERY important for using the Dynamic Topic Modeling later on. 

# final_df = text_content_df.sort_values(['Year'], ascending=True).reset_index(drop=True)
# final_df.head(15)

## Resources
## https://stackoverflow.com/questions/53332116/reset-index-after-sorting-data-frame

In [ ]:
## Add a unique identifier to each document in the dataset

## If following the remove duplicates workflow, use this code:
# final_df['unique_id']=final_df.index
# final_df.head(15)

## If you did not use the "remove duplicates" workflow, use this code:

text_df['unique_id']=text_df.index
final_df = text_df
final_df.head(15)

## Resources
## https://stackoverflow.com/questions/44878740/how-do-i-create-a-unique-record-id-in-a-python-dataframe
## df['unique_id'] = df.longstrings.map(hash)

In [ ]:
## Remove "new line" separater (usually 'r\n\') so as to not create a content column where the text breaks into new rows.
## You will notice that some lines do break in Excel, but this is a problem with excel and not with the CSV. 

final_df = final_df.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\r\n',  ' ', regex=True)
final_df.head(15)

## Resource(s)
## https://stackoverflow.com/questions/46522652/replacing-newlines-with-spaces-for-str-columns-through-pandas-dataframe


In [ ]:
## Save dataframe to csv
with open(r"output/loading/final_df.csv", 'w', encoding='utf-8') as file:
    final_df.to_csv(file, line_terminator='\n', index=True)
    ##  
    file.close()

In [ ]:
## Create a list of the unique IDS
identifiers = final_df['unique_id'].tolist()
print(identifiers)

In [ ]:
## Save the list using pkl

file_name = "output/loading/identifiers.pkl"

open_file = open(file_name, "wb")
pickle.dump(identifiers, open_file, protocol=4)
open_file.close()

